In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Dissertation')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
import numpy as np

In [ ]:
# Countries code
# USA, China, Japan, Germany, UK, France, India, Italy, Canada, Korea
countries = ['842', '156', '392', '276', '826', '251', '699', '381', '124', '410']

In [ ]:
# Commodity classifier (2 digit Hamonized System codes)
# Animal & Animal Products & Raw Hides, Skins, Leather & Furs 
c_1 = ['01','02','03','04','05','41','42','43']
# Vegetable Products & Foodstuffs
c_2 = ['06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']
# Mineral Products
c_3 = ['25','26','27']
# Chemicals & Allied Industries
c_4 = ['28','29','30','31','32','33','34','35','36','37','38']
# Plastics & Rubbers
c_5 = ['39','40']
# Textiles
c_6 = ['50','51','52','53','54','55','56','57','58','59','60','61','62','63']
# Metals
c_7 = ['72','73','74','75','76','77','78','79','80','81','82','83']
# Machinery & Electrical
c_8 = ['84','85']
# total_class
total_class = [c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8]

In [ ]:
# Write a function that read data into well structured form
# input : data from one request (one year, from one expoter to one importer)
# output : 12*8 tensor
def read_data(data, year):
  output = np.zeros(12*8).reshape(12, 8)
  if data['dataset'] == []:
    return output
  else:
    for i in range(len(data['dataset'])):
      z = data['dataset'][i]['period'] - year - 1
      for index, classifier in enumerate(total_class):
        if data['dataset'][i]['cmdCode'] in classifier:
          output[z, index] += data['dataset'][i]['TradeValue']
    return output        

In [ ]:
# Write a function that request data 10*10 (year * importer) times
def request_data(exporter):
  result = np.zeros(120*10*8).reshape(120, 10, 8)
  years = [201000, 201100, 201200, 201300, 201400, 201500, 201600, 201700, 201800, 201900]
  periods = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']
  countries = ['842', '156', '392', '276', '826', '251', '699', '381', '124', '410']
  count = 0
  for index, year in enumerate(years):
    for country_index, country in enumerate(countries):
      # request data
      count += 1
      start = 'http://comtrade.un.org/api/get?max=10000&type=C&freq=M&px=HS&ps='
      url = start + periods[index]+ '&r=' + exporter + '&p='+ country + '&rg=2&cc=AG2'
      response = requests.get(url)
      status = response.status_code
      if status == 200:
        print(count)
        get_data = response.json()
        start_time = index*12
        end_time = start_time + 12
        result[start_time:end_time, country_index, :] = read_data(data=get_data, year=year)
      else:
        print('Count: ', count)
        print('Bad Data from Server: ', status)
  return result

In [ ]:
# USA
data_1 = request_data('842')

In [ ]:
# China
data_2 = request_data('156')

In [ ]:
# Japan
data_3 = request_data('392')

In [ ]:
# Germany
data_4 = request_data('276')

In [ ]:
# UK
data_5 = request_data('826')

In [ ]:
# France
data_6 = request_data('251')

In [ ]:
# India
data_7 = request_data('699')

In [ ]:
# Italy
data_8 = request_data('381')

In [ ]:
# Canada
data_9 = request_data('124')

In [ ]:
# Korea
data_10 = request_data('410')

In [ ]:
# Fill missing data
# Countries code
# USA, China, Japan, Germany, UK, France, India, Italy, Canada, Korea
countries = ['842', '156', '392', '276', '826', '251', '699', '381', '124', '410']
output = np.zeros(12*8).reshape(12,8)
url = 'http://comtrade.un.org/api/get?max=10000&type=C&freq=M&px=HS&ps=2014&r=699&p=410&rg=1&cc=AG2'
response = requests.get(url)
status = response.status_code
print(status)
data = response.json()
if data['dataset'] == []:
  print("No data")
else:
  output =  read_data(data, 201400)
  print(output)